### 1. UIautomation库

> https://github.com/yinkaisheng/Python-UIAutomation-for-Windows/blob/master/readme_cn.md

In [1]:
# 测试UIautomation代码
# -*- coding: utf-8 -*-
import subprocess
import uiautomation as auto

def test():
    print(auto.GetRootControl())
    subprocess.Popen('notepad.exe')
    # 首先从桌面的第一层子控件中找到记事本程序的窗口WindowControl，再从这个窗口查找子控件
    notepadWindow = auto.WindowControl(searchDepth=1, ClassName='Notepad')
    print(notepadWindow.Name)
    notepadWindow.SetTopmost(True)
    # 查找notepadWindow所有子孙控件中的第一个EditControl，因为EditControl是第一个子控件，可以不指定深度
    edit = notepadWindow.EditControl()
    try:
        # 获取EditControl支持的ValuePattern，并用Pattern设置控件文本为"Hello"
        edit.GetValuePattern().SetValue('Hello')# or edit.GetPattern(auto.PatternId.ValuePattern)
    except auto.comtypes.COMError as ex:
        # 如果遇到COMError, 一般是没有以管理员权限运行Python, 或者这个控件没有实现pattern的方法(如果是这种情况，基本没有解决方法)
        # 大多数情况不需要捕捉COMError，如果遇到了就加到try block
        pass
    edit.SendKeys('{Ctrl}{End}{Enter}World')# 在文本末尾打字
    print('current text:', edit.GetValuePattern().Value)# 获取当前文本
    # 先从notepadWindow的第一层子控件中查找TitleBarControl, 
    # 然后从TitleBarControl的子孙控件中找第二个ButtonControl, 即最大化按钮，并点击按钮
    notepadWindow.TitleBarControl(Depth=1).ButtonControl(foundIndex=2).Click()
    # 从notepadWindow前两层子孙控件中查找Name为'关闭'的按钮并点击按钮
    notepadWindow.ButtonControl(searchDepth=2, Name='关闭').Click()
    # 这时记事本弹出是否保存提示，按热键Alt+N不保存退出。
    auto.SendKeys('{Alt}n')

if __name__ == '__main__':
    test()

ControlType: PaneControl    ClassName: #32769    AutomationId:     Rect: (0,0,1920,1080)[1920x1080]    Name: '桌面 1'    Handle: 0x10010(65552)
无标题 - 记事本
current text: Hello
World


# 2. 微信窗口

双击聊天人弹出单独的窗口后

In [4]:
# wechat
import uiautomation
import time

"""
# TODO 用正则表达式提取文本内需要的信息
import re
def to_list_table(txt_msg:str, path="./date/xxx.csv"):
    table = []
    ? uid = re.findall("(\d{17,}(\d|X|x))",txt_msg)
    ? uid = re.findall("(\d{17,}[\dXx])",txt_msg)
    ......
    table.apend([uid,ct,time,lab_name...])
    with open(path) as f:
        f.write(table, 'a', encoding='utf-8_sig')
        for i in table:
            send_text_msg(f"身份证{i[0]},已经存档", trans_window_name="xxxx群")
"""


# 定位微信主窗口，获取窗口句柄
spying_window_name = "文件传输助手"
spying_window = uiautomation.WindowControl(searchDepth=1, Name=spying_window_name)

def get_msg_list():
    msg_list = [] 
    # 找到窗口的list部件然后获取内容
    msg_ctrl_list = spying_window.ListControl().GetChildren()
    # print(f" Test1: the first msg_ctrl is : {msg_ctrl_list[0]} \n Test2: the last msg_ctrl is : {msg_ctrl_list[-1]} \n Test3: the length of msg_List is : {len(msg_ctrl_list)} ") #
    for msg_ctrl in msg_ctrl_list:
        msg_list.append(msg_ctrl.Name)
        # print(f" Test0: the msg_ctrl is : {msg_ctrl}")  
        # "Test0: the msg_ctrl is : ControlType: ListItemControl    ClassName:     AutomationId:     Rect: (993,-3612,1543,-3578)[550x34]    Name: '星期天 22:40'    Handle: 0x0(0)"
        # ......
        # "Test0: the msg_ctrl is : ControlType: ListItemControl    ClassName:     AutomationId:     Rect: (993,-2237,1543,-2106)[550x131]    Name: '[文件]'    Handle: 0x0(0)"
        # ? 排除文本以外内容 Name's category: '查看更多消息', '星期天 22:40', '[动画表情]', '[文件]', '[图片]', '昨天 19:22', '[OK]', '20:14', '字符串信息之类的东西'; 
    # print(f" Test4: msg_list is : {msg_list}") #
    return msg_list

def send_text_msg(text_msg:str, trans_window_name:str):
    # 双击得到单独窗口的需要转发的群
    trans_window = uiautomation.WindowControl(searchDepth=1, Name=trans_window_name)
    trans_window.SendKeys(text_msg) # 写入消息
    trans_window.SendKey(13) # 点击Enter按键ASCⅡ

former_msg_list = get_msg_list() # 上一消息列表状态消息
while True:
    latest_msg_list = get_msg_list()
    if latest_msg_list[-1] != former_msg_list[-1]: # 末条消息的Name和上一条消息的Name不同
        new_msg = latest_msg_list[-1]
        print(new_msg , f"最后一次列表长度{len(latest_msg_list)}")
        # send_text_msg(new_msg)
        former_msg_list = get_msg_list()
        time.sleep(3)

KeyboardInterrupt: 

# 2.查看所有窗口句柄和名称

In [5]:
import win32gui

hwnd_title = dict()

def get_all_hwnd(hwnd, mouse):
    if win32gui.IsWindow(hwnd) and win32gui.IsWindowEnabled(hwnd) and win32gui.IsWindowVisible(hwnd):
        hwnd_title.update({hwnd: win32gui.GetWindowText(hwnd)})

win32gui.EnumWindows(get_all_hwnd, 0)    

for h, t in hwnd_title.items():
    if t != "":
        print(h, t)

67862 QQ
66494 小火箭托盘加速
197656 SCM
657810 ● UIautomation-1.ipynb - Visual Studio Code
853572 [python]微信自动回复-通用窗口操作接口uiautomation_哔哩哔哩_bilibili — Mozilla Firefox
8129044 伊卡酱的播种船等3个会话
3739266 文件传输助手
3278926 微信
132036 NVIDIA GeForce Overlay
131162 Microsoft Text Input Application
197978 设置
66840 设置
67158 Killer Intelligence Center
196830 Program Manager
